## 1. Imports


In [61]:
import os
import sys
sys.path.append('../')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns;

In [76]:
data_path = '../results/baseline_std100_m75,150/'
folders= ['dt','gnb','lgr','gbt']
#folders= ['dt']

## 2. CSVs

In [77]:
def impact_csvs(data_path= 'data/results/',b_or_w = 'Black', folders= ['dt','gnb','lgr','gbt']):

    col_names_df = []

    for i,f in enumerate(folders):
        if b_or_w == 'Black':
            path = f'{data_path}{f}/{f}_black_results.csv'
        else:
            path = f'{data_path}{f}/{f}_white_results.csv'

        df = pd.read_csv(path,index_col=0)
        df = df.reset_index()
        
        col_names_df.append(f'{f.upper()}')

        if i == 0:
            joined_df = df.iloc[:,-1]
        else:
            joined_df = pd.concat([joined_df, df.iloc[:,-1]], axis=1)

    joined_df.set_axis(folders, axis=1)

    # split dataframe after the two reduction algorithms
    df = joined_df.iloc[:6,:]

    # set new index
    df['Constraint'] = ['Unmitigated', 'DP', 'EO', 'EOO','FPER','ERP']
    df.set_index('Constraint',inplace=True)
    
    df.columns = col_names_df

    print('Group: ',b_or_w,'\n DataFrame: \n',df)
    df.to_csv(f'{data_path}/{b_or_w}_DI.csv')


In [78]:
impact_csvs(data_path,'Black', folders= ['dt','gnb','lgr','gbt'])
impact_csvs(data_path,'White', folders= ['dt','gnb','lgr','gbt'])

Group:  Black 
 DataFrame: 
                 DT    GNB    LGR    GBT
Constraint                             
Unmitigated   4.16   0.00   3.04   4.12
DP          -36.01 -22.98 -35.59 -33.01
EO           -7.54 -14.90  -8.23  -6.90
EOO         -14.78 -19.36 -16.22 -15.00
FPER         -8.52 -16.98  -9.52  -7.74
ERP           4.82   6.76   4.37   5.09
Group:  White 
 DataFrame: 
                 DT    GNB    LGR    GBT
Constraint                             
Unmitigated  25.79  24.81  26.01  26.81
DP           25.90  24.59  26.26  26.57
EO           25.99  24.10  25.36  25.81
EOO          26.56  26.59  26.18  26.35
FPER         26.30  26.03  26.78  25.64
ERP          23.20  22.15  25.58  25.07


C:\Users\HaRi\AppData\Local\Temp\ipykernel_21704\2563935939.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Constraint'] = ['Unmitigated', 'DP', 'EO', 'EOO','FPER','ERP']
C:\Users\HaRi\AppData\Local\Temp\ipykernel_21704\2563935939.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Constraint'] = ['Unmitigated', 'DP', 'EO', 'EOO','FPER','ERP']


In [79]:
def neg_impact_csvs(data_path= 'data/results/',b_or_w = 'Black', folders= ['dt','gnb','lgr','gbt']):

    col_names_df = []

    for i,f in enumerate(folders):
        if b_or_w == 'Black':
            path = f'{data_path}{f}/{f}_type_ratios.csv'
            df = pd.read_csv(path,index_col=0)
            df = df.filter(like='B')          

        else:
            path = f'{data_path}{f}/{f}_type_ratios.csv'
            df = pd.read_csv(path,index_col=0)
            df = df.filter(like='W')          
        col_names_df.append(f'{f.upper()}')
        df = df.iloc[3,[6,0,1,5,3,2]] - df.iloc[0,[6,0,1,5,3,2]]
    
        if i == 0:
            joined_df = df.iloc[:]
        else:
            joined_df = pd.concat([joined_df, df.iloc[:]], axis=1)

    joined_df.set_axis(folders, axis=1)
    
    # split dataframe after the two reduction algorithms
    df = joined_df.iloc[:6,:]

    # set new index
    df['Constraint'] = ['Unmitigated', 'DP', 'EO', 'EOO','FPER','ERP']
    df.set_index('Constraint',inplace=True)
    
    df.columns = col_names_df

    print('Group: ',b_or_w,'\n DataFrame: \n',df)

    df.to_csv(f'{data_path}/{b_or_w}_FN_I.csv')



## 3. FP/TP/TN/FN Ratios

In [80]:
# Ratios
dfs = {} # list for pandas dfs
pd.set_option('display.max_columns', None)
for i,f in enumerate(folders):
    path = f'{data_path}{f}/{f}_all_types.csv'
    df = pd.read_csv(path)
    df = df.reset_index(drop=True)
    df = df.melt(var_name="ID",value_name="Category")
    df = df.groupby('ID').value_counts(normalize=True)
    df = df.reset_index()
    df = df.rename(columns= {0:'Ratio'})
    
    df = df.pivot(index='Category', columns='ID')['Ratio']
    df = df.fillna(0)
    df = df.round(decimals = 3)
    df.to_csv(f'{data_path}{f}/{f}_type_ratios.csv')

In [81]:
neg_impact_csvs(data_path,'Black', folders= ['dt','gnb','lgr','gbt'])
neg_impact_csvs(data_path,'White', folders= ['dt','gnb','lgr','gbt'])

Group:  Black 
 DataFrame: 
                 DT    GNB    LGR    GBT
Constraint                             
Unmitigated  0.149 -0.337  0.137  0.157
DP           0.321  0.239  0.321  0.319
EO           0.235  0.259  0.245  0.235
EOO          0.259  0.279  0.275  0.265
FPER         0.238  0.047  0.243  0.238
ERP          0.149  0.071  0.134  0.153
Group:  White 
 DataFrame: 
                 DT    GNB    LGR    GBT
Constraint                             
Unmitigated  0.686  0.705  0.687  0.684
DP           0.665  0.562  0.665  0.662
EO           0.598  0.627  0.602  0.588
EOO          0.644  0.667  0.655  0.650
FPER         0.662  0.673  0.670  0.662
ERP          0.638  0.613  0.569  0.603


C:\Users\HaRi\AppData\Local\Temp\ipykernel_21704\724994539.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Constraint'] = ['Unmitigated', 'DP', 'EO', 'EOO','FPER','ERP']
C:\Users\HaRi\AppData\Local\Temp\ipykernel_21704\724994539.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Constraint'] = ['Unmitigated', 'DP', 'EO', 'EOO','FPER','ERP']


In [82]:
# Absolute numbers
dfs = {} # list for pandas dfs
pd.set_option('display.max_columns', None)
for i,f in enumerate(folders):
    path = f'{data_path}{f}/{f}_all_types.csv'
    df = pd.read_csv(path)
    df = df.reset_index(drop=True)
    df = df.melt(var_name="ID",value_name="Category")
    df = df.groupby('ID').value_counts()
    df = df.reset_index()
    df = df.rename(columns= {0:'Number'})
    
    df = df.pivot(index='Category', columns='ID')['Number']
    df = df.fillna(0)
    df.to_csv(f'{data_path}{f}/{f}_type_absolute.csv')

# Analyzing Scores

#### Extractig Scores from csv into dataframes

In [83]:
# Scores Data Frames
classifier_dfs = {}
dfs_b = {}
dfs_w = {}

for f in folders:
    path = f'{data_path}{f}/{f}_all_scores.csv'
    df = pd.read_csv(path)
    df = df.reset_index(drop=True)
    df = df.round(0)

    df_black = df.filter(like='B')
    df_white = df.filter(like='W')
    
    classifier_dfs[f] = df
    dfs_b[f] = df_black
    dfs_w[f] = df_white

### Checking if normal distributions:

if p < 0.01 (or < 0.05) then the distribution is significantly different from a normal distribution

In [84]:
for c,df in classifier_dfs.items():
    print('Classifier:',c)
    for col in df:
        data=df[col].dropna(axis=0)
        _,p = stats.kstest(data, "norm")
        if p > 0.01:
            print(col,',p:',p)
    print('Check for norm Distributions done')

Classifier: dt
Check for norm Distributions done
Classifier: gnb
Check for norm Distributions done
Classifier: lgr
Check for norm Distributions done
Classifier: gbt
Check for norm Distributions done


### Mann Whitney U test:

“a two-sample rank test for the difference between two population medians . . . It assumes that the data are independent random samples from two populations that have the same shape.”

In [85]:
mwu_path = f'{data_path}mwu/'
os.makedirs(mwu_path,exist_ok = True)

#### Variance of Distributions Each Model against each other
if p < 0.001 (or < 0.05) then the distributions are significantly different from each other

In [86]:
def p_values_mwu_1model(dfs,model=''):
    p_vals = pd.DataFrame(data={'Classifier': []})
    p_signi = pd.DataFrame(data={'Classifier': []})
    
    for c1,df1 in dfs.items():
        col_signi = []
        col_vals = []
        idx = []
        
        data1 = df1[model].dropna(axis=0)
        
        for c2,df2 in dfs.items():
            idx.append(c2)
            
            data2 =df2[model].dropna(axis=0)
            
            _,p = stats.mannwhitneyu(data1, data2)
            
            col_vals.append(p)
            if p< 0.05:
                col_signi.append('s')
            else:
                col_signi.append(' ')
                
        p_signi[c1] = col_signi
        p_vals[c1] = col_vals
        
    p_vals['Classifier'] = idx
    p_vals = p_vals.set_index('Classifier')
    p_signi['Classifier'] = idx
    p_signi = p_signi.set_index('Classifier')
    
    p_vals = p_vals.round(decimals=3)
    print(p_signi)
    
    p_vals.to_csv(f'{mwu_path}p_{model}.csv')
    p_signi.to_csv(f'{mwu_path}significanz_{model}.csv')

In [87]:
for col in classifier_dfs['dt']:
    print('\nC:',col)
    p_values_mwu_1model(classifier_dfs,col)


C: testB
           dt gnb lgr gbt
Classifier               
dt                       
gnb                      
lgr                      
gbt                      

C: testW
           dt gnb lgr gbt
Classifier               
dt                       
gnb                      
lgr                      
gbt                      

C: unmitB
           dt gnb lgr gbt
Classifier               
dt              s        
gnb         s       s   s
lgr             s        
gbt             s        

C: unmitW
           dt gnb lgr gbt
Classifier               
dt                       
gnb                      
lgr                      
gbt                      

C: dpB
           dt gnb lgr gbt
Classifier               
dt              s        
gnb         s       s   s
lgr             s        
gbt             s        

C: dpW
           dt gnb lgr gbt
Classifier               
dt                       
gnb                      
lgr                      
gbt                      

C: eo

#### Variance of Distributions unmitigated v Mitigated for each race

if p < 0.001 (or < 0.0005) then the distributions are significantly different from each other

In [88]:
def p_race_mwu(dfs, b_or_w = 'B'):
    p_vals = pd.DataFrame(data={'Constraints': []})
    p_signi = pd.DataFrame(data={'Constraints': []})
    
    for c,df in dfs.items():
        
        c = f'{c}{b_or_w}'
        col_signi = []
        col_vals = []
        idx = []
        
        data_unmiti = df[f'unmit{b_or_w}'].dropna(axis=0)
        df = df.iloc[:,1:]
        for col in df:
            
            idx.append(col[:-1])
            
            data_miti=df[col].dropna(axis=0)
            
            _,p = stats.mannwhitneyu(data_unmiti, data_miti)
            
            col_vals.append(p)
            if p< 0.05:
                col_signi.append('s')
            else:
                col_signi.append(' ')
                
        p_signi[c] = col_signi
        p_vals[c] = col_vals
        
    p_vals['Constraints'] = idx
    p_vals = p_vals.set_index('Constraints')
    
    
    
    p_signi['Constraints'] = idx
    p_signi = p_signi.set_index('Constraints')
    
    p_vals = p_vals.round(decimals=3)
    print(p_signi)
    
    p_vals.to_csv(f'{mwu_path}p_un_vs_miti_{b_or_w}.csv')
    p_signi.to_csv(f'{mwu_path}significanz_un_vs_miti_{b_or_w}.csv')
   

In [89]:
print('Black:')
p_race_mwu(dfs_b,'B')

print('\nWhite:')
p_race_mwu(dfs_w,'W')

Black:
            dtB gnbB lgrB gbtB
Constraints                   
unmit                         
dp            s    s    s    s
eo            s    s    s    s
tprp          s    s    s    s
fprp          s    s    s    s
erp                           

White:
            dtW gnbW lgrW gbtW
Constraints                   
unmit                         
dp                            
eo                 s          
tprp                          
fprp                          
erp                s          
